In [1]:
import oyaml as yaml

import io

import boto3

import json

In [38]:
import os

In [33]:
a={
            "env": "dev",
            "app": "binance-gateway"
        }
json.dumps(a)

'{"env": "dev", "app": "binance-gateway"}'

In [2]:
session = boto3.session.Session(profile_name="bijietech-tools")

In [26]:
dev_session = boto3.session.Session(profile_name="bijietech-dev")
dev_ec2 = dev_session.client("ec2")
dev_asg = dev_session.client("autoscaling")

In [28]:
res = dev_asg.describe_auto_scaling_groups(AutoScalingGroupNames=["tf_bin_dev_binance_gateway_ec2_asg"])

In [32]:
len(res["AutoScalingGroups"][0]["Instances"])

[{'InstanceId': 'i-070e7c9fa6abf6f48',
  'InstanceType': 'c5.xlarge',
  'AvailabilityZone': 'ap-northeast-1a',
  'LifecycleState': 'InService',
  'HealthStatus': 'Healthy',
  'LaunchTemplate': {'LaunchTemplateId': 'lt-08cf3be957176fe47',
   'LaunchTemplateName': 'tf_bin_dev_binance_gateway_ec2_lt',
   'Version': '12'},
  'ProtectedFromScaleIn': True},
 {'InstanceId': 'i-0bd6945ec0d770163',
  'InstanceType': 'm5.xlarge',
  'AvailabilityZone': 'ap-northeast-1c',
  'LifecycleState': 'InService',
  'HealthStatus': 'Healthy',
  'LaunchTemplate': {'LaunchTemplateId': 'lt-08cf3be957176fe47',
   'LaunchTemplateName': 'tf_bin_dev_binance_gateway_ec2_lt',
   'Version': '11'},
  'ProtectedFromScaleIn': False}]

In [5]:
sts = session.client('sts')

In [6]:
sts.get_caller_identity()

{'UserId': 'AROA5LUC6QITJVTXVVFP4:desmond.cao@binance.com',
 'Account': '918323888678',
 'Arn': 'arn:aws:sts::918323888678:assumed-role/desmond.cao/desmond.cao@binance.com',
 'ResponseMetadata': {'RequestId': 'f2049e31-f7e7-4db6-a7b3-368a5830eb48',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f2049e31-f7e7-4db6-a7b3-368a5830eb48',
   'content-type': 'text/xml',
   'content-length': '464',
   'date': 'Tue, 04 Apr 2023 02:55:46 GMT'},
  'RetryAttempts': 0}}

In [8]:
# Assume the role
assumed_role = sts.assume_role(
    RoleArn='arn:aws:iam::173062506398:role/mainsite_dev_binance_gateway_helper',
    RoleSessionName='test'
)

In [9]:
dev_session = boto3.Session(
    aws_access_key_id=assumed_role['Credentials']['AccessKeyId'],
    aws_secret_access_key=assumed_role['Credentials']['SecretAccessKey'],
    aws_session_token=assumed_role['Credentials']['SessionToken']
)

In [10]:
dev_s3 = dev_session.client("s3")

In [15]:
files = dev_s3.list_objects_v2(Bucket="tk-dev-cicd", Prefix="backend/binance-gateway/")["Contents"]

In [18]:
files[0]

{'Key': 'backend/binance-gateway/-tesla-gateway-1.0.0.jar',
 'LastModified': datetime.datetime(2021, 12, 7, 8, 2, 45, tzinfo=tzutc()),
 'ETag': '"adf9482304a4fbf1b15b16eda040ad89-15"',
 'Size': 118627958,
 'StorageClass': 'STANDARD'}

In [23]:
files.sort(key=lambda one: one["LastModified"], reverse=True)

In [34]:
files

[{'Key': 'backend/binance-gateway/tesla-gateway-1.0.0.jar',
  'LastModified': datetime.datetime(2023, 3, 22, 8, 43, 32, tzinfo=tzutc()),
  'ETag': '"22667ea5e05564c558433c31005e2dbe-16"',
  'Size': 128106488,
  'StorageClass': 'STANDARD'},
 {'Key': 'backend/binance-gateway/221-e83eb2c64-20230322084238-tesla-gateway-1.0.0.jar',
  'LastModified': datetime.datetime(2023, 3, 22, 8, 43, 28, tzinfo=tzutc()),
  'ETag': '"22667ea5e05564c558433c31005e2dbe-16"',
  'Size': 128106488,
  'StorageClass': 'STANDARD'},
 {'Key': 'backend/binance-gateway/220-e83eb2c64-20230322082518-tesla-gateway-1.0.0.jar',
  'LastModified': datetime.datetime(2023, 3, 22, 8, 26, 17, tzinfo=tzutc()),
  'ETag': '"e11fb26535d61b07be406abba942cc04-16"',
  'Size': 128106489,
  'StorageClass': 'STANDARD'},
 {'Key': 'backend/binance-gateway/218-96de4b69e-20230214092022-tesla-gateway-1.0.0.jar',
  'LastModified': datetime.datetime(2023, 2, 14, 9, 21, 16, tzinfo=tzutc()),
  'ETag': '"904cd79882f587900257fdd3fcae672a-16"',
  'Si

In [35]:
files.sort(key=lambda one: one["LastModified"], reverse=True)

In [39]:
jar_names = list(map(lambda one: os.path.basename(one["Key"]), files[:10]))

In [40]:
jar_names

['tesla-gateway-1.0.0.jar',
 '221-e83eb2c64-20230322084238-tesla-gateway-1.0.0.jar',
 '220-e83eb2c64-20230322082518-tesla-gateway-1.0.0.jar',
 '218-96de4b69e-20230214092022-tesla-gateway-1.0.0.jar',
 '217-96de4b69e-20230214082519-tesla-gateway-1.0.0.jar',
 '216-96de4b69e-20230214080714-tesla-gateway-1.0.0.jar',
 '214-96de4b69e-20230214075043-tesla-gateway-1.0.0.jar',
 '213-96de4b69e-20230214073932-tesla-gateway-1.0.0.jar',
 '212-96de4b69e-20230214072702-tesla-gateway-1.0.0.jar',
 '211-96de4b69e-20230213085527-tesla-gateway-1.0.0.jar']

In [6]:
ec2 = session.client("ec2")

In [7]:
ecs = session.client("ecs")

In [8]:
alb = session.client("elbv2")

In [10]:
res = alb.describe_load_balancers(Names=["bin-dev-desmond-debug-alb"])

In [11]:
res

{'LoadBalancers': [{'LoadBalancerArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:173062506398:loadbalancer/app/bin-dev-desmond-debug-alb/c35e4fc77a40cf0c',
   'DNSName': 'internal-bin-dev-desmond-debug-alb-718339714.ap-northeast-1.elb.amazonaws.com',
   'CanonicalHostedZoneId': 'Z14GRHDCWA56QT',
   'CreatedTime': datetime.datetime(2021, 8, 12, 8, 27, 28, 480000, tzinfo=tzutc()),
   'LoadBalancerName': 'bin-dev-desmond-debug-alb',
   'Scheme': 'internal',
   'VpcId': 'vpc-06e7bab377f9cb4e9',
   'State': {'Code': 'active'},
   'Type': 'application',
   'AvailabilityZones': [{'ZoneName': 'ap-northeast-1d',
     'SubnetId': 'subnet-005223e14aa168a2c',
     'LoadBalancerAddresses': []},
    {'ZoneName': 'ap-northeast-1a',
     'SubnetId': 'subnet-0e5d061593dd742a0',
     'LoadBalancerAddresses': []},
    {'ZoneName': 'ap-northeast-1c',
     'SubnetId': 'subnet-0f7c148c34aff7ba0',
     'LoadBalancerAddresses': []}],
   'SecurityGroups': ['sg-08bef9f168dcbafba', 'sg-0775fd1862783c47e'],
  

In [ ]:
# sharesvc-logansite-service
# sharesvc-loganserver-service

In [44]:
ecs.describe_services(cluster="bnb-ocbs-sharevpc", services=["sharesvc-loganserver-service"])["services"][0]

{'serviceArn': 'arn:aws:ecs:ap-northeast-1:369096419186:service/sharesvc-loganserver-service',
 'serviceName': 'sharesvc-loganserver-service',
 'clusterArn': 'arn:aws:ecs:ap-northeast-1:369096419186:cluster/bnb-ocbs-sharevpc',
 'loadBalancers': [{'targetGroupArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logan-server-tg/7f3592cb6275b4b3',
   'containerName': 'sharesvc-loganserver',
   'containerPort': 8080},
  {'targetGroupArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-loganserver-p-tg/fb3441a6fab3b3af',
   'containerName': 'sharesvc-loganserver',
   'containerPort': 8080}],
 'serviceRegistries': [],
 'status': 'ACTIVE',
 'desiredCount': 1,
 'runningCount': 1,
 'pendingCount': 0,
 'launchType': 'FARGATE',
 'platformVersion': 'LATEST',
 'platformFamily': 'Linux',
 'taskDefinition': 'arn:aws:ecs:ap-northeast-1:369096419186:task-definition/sharesvc-loganserver:28',
 'deploymentConfiguration': {'deploymentCircuitBr

In [7]:
res = ecs.describe_services(cluster="bnb-ocbs-sharevpc", services=["sharesvc-logansite-service"])

In [10]:
res["services"][0]

{'serviceArn': 'arn:aws:ecs:ap-northeast-1:369096419186:service/sharesvc-logansite-service',
 'serviceName': 'sharesvc-logansite-service',
 'clusterArn': 'arn:aws:ecs:ap-northeast-1:369096419186:cluster/bnb-ocbs-sharevpc',
 'loadBalancers': [{'targetGroupArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logansite-tg/0170f6bb3fbe6595',
   'containerName': 'sharesvc-logansite',
   'containerPort': 8080}],
 'serviceRegistries': [],
 'status': 'ACTIVE',
 'desiredCount': 1,
 'runningCount': 1,
 'pendingCount': 0,
 'launchType': 'FARGATE',
 'platformVersion': 'LATEST',
 'platformFamily': 'Linux',
 'taskDefinition': 'arn:aws:ecs:ap-northeast-1:369096419186:task-definition/sharesvc-logansite:17',
 'deploymentConfiguration': {'deploymentCircuitBreaker': {'enable': False,
   'rollback': False},
  'maximumPercent': 200,
  'minimumHealthyPercent': 100},
 'deployments': [{'id': 'ecs-svc/8161219552491274083',
   'status': 'PRIMARY',
   'taskDefinition': 'arn:aws:ec

In [45]:
alb.describe_target_groups(TargetGroupArns=["arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logan-server-tg/7f3592cb6275b4b3"])["TargetGroups"][0]

{'TargetGroupArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logan-server-tg/7f3592cb6275b4b3',
 'TargetGroupName': 'sharesvc-logan-server-tg',
 'Protocol': 'HTTP',
 'Port': 8080,
 'VpcId': 'vpc-026ddb1dc52fc3b25',
 'HealthCheckProtocol': 'HTTP',
 'HealthCheckPort': 'traffic-port',
 'HealthCheckEnabled': True,
 'HealthCheckIntervalSeconds': 20,
 'HealthCheckTimeoutSeconds': 5,
 'HealthyThresholdCount': 2,
 'UnhealthyThresholdCount': 2,
 'HealthCheckPath': '/logan/meta/logtypes.json',
 'Matcher': {'HttpCode': '200'},
 'LoadBalancerArns': ['arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5'],
 'TargetType': 'ip',
 'ProtocolVersion': 'HTTP1',
 'IpAddressType': 'ipv4'}

In [13]:
tg = alb.describe_target_groups(TargetGroupArns=["arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logansite-tg/0170f6bb3fbe6595"])

In [15]:
tg["TargetGroups"][0]

{'TargetGroupArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logansite-tg/0170f6bb3fbe6595',
 'TargetGroupName': 'sharesvc-logansite-tg',
 'Protocol': 'HTTP',
 'Port': 8080,
 'VpcId': 'vpc-026ddb1dc52fc3b25',
 'HealthCheckProtocol': 'HTTP',
 'HealthCheckPort': 'traffic-port',
 'HealthCheckEnabled': True,
 'HealthCheckIntervalSeconds': 30,
 'HealthCheckTimeoutSeconds': 5,
 'HealthyThresholdCount': 2,
 'UnhealthyThresholdCount': 2,
 'HealthCheckPath': '/',
 'Matcher': {'HttpCode': '200'},
 'LoadBalancerArns': ['arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-private-alb/9c82d15debe62f2b'],
 'TargetType': 'ip',
 'ProtocolVersion': 'HTTP1',
 'IpAddressType': 'ipv4'}

In [48]:
json.dumps(ecs.describe_task_definition(taskDefinition="sharesvc-loganserver")["taskDefinition"], default=str)

'{"taskDefinitionArn": "arn:aws:ecs:ap-northeast-1:369096419186:task-definition/sharesvc-loganserver:28", "containerDefinitions": [{"name": "sharesvc-loganserver", "image": "369096419186.dkr.ecr.ap-northeast-1.amazonaws.com/logan-server:731f8efb3058d5e0cf864a46aface842fd26647d", "cpu": 4096, "memory": 8192, "portMappings": [{"containerPort": 8080, "hostPort": 8080, "protocol": "tcp"}], "essential": true, "environment": [], "mountPoints": [{"sourceVolume": "myEfsVolume", "containerPath": "/data"}], "volumesFrom": [], "ulimits": [{"name": "nofile", "softLimit": 65535, "hardLimit": 65535}], "logConfiguration": {"logDriver": "awslogs", "options": {"awslogs-datetime-format": "%d/%b/%Y:%T", "awslogs-group": "/bin-prod-ecs", "awslogs-region": "ap-northeast-1", "awslogs-stream-prefix": "/sharesvc-loganserver-service"}}}], "family": "sharesvc-loganserver", "executionRoleArn": "arn:aws:iam::369096419186:role/ecsTaskExecutionRole", "networkMode": "awsvpc", "revision": 28, "volumes": [{"name": "my

In [18]:
td = ecs.describe_task_definition(taskDefinition="sharesvc-logansite")

In [20]:
td['taskDefinition']

{'taskDefinitionArn': 'arn:aws:ecs:ap-northeast-1:369096419186:task-definition/sharesvc-logansite:17',
 'containerDefinitions': [{'name': 'sharesvc-logansite',
   'image': '369096419186.dkr.ecr.ap-northeast-1.amazonaws.com/logan-site:202302151056',
   'cpu': 2048,
   'memory': 4096,
   'portMappings': [{'containerPort': 8080,
     'hostPort': 8080,
     'protocol': 'tcp'}],
   'essential': True,
   'environment': [],
   'mountPoints': [],
   'volumesFrom': [],
   'ulimits': [{'name': 'nofile', 'softLimit': 65535, 'hardLimit': 65535}],
   'logConfiguration': {'logDriver': 'awslogs',
    'options': {'awslogs-datetime-format': '%d/%b/%Y:%T',
     'awslogs-group': '/bin-prod-ecs',
     'awslogs-region': 'ap-northeast-1',
     'awslogs-stream-prefix': '/sharesvc-logansite-service'}}}],
 'family': 'sharesvc-logansite',
 'executionRoleArn': 'arn:aws:iam::369096419186:role/ecsTaskExecutionRole',
 'networkMode': 'awsvpc',
 'revision': 17,
 'volumes': [],
 'status': 'ACTIVE',
 'requiresAttribute

In [25]:
sgs = ec2.describe_security_groups(GroupIds=["sg-04df0e7fdf83a8632"])

In [27]:
sgs["SecurityGroups"][0]

{'Description': 'sharesvc-logansite-task-sg',
 'GroupName': 'sharesvc-logansite-task-sg',
 'IpPermissions': [{'FromPort': 8080,
   'IpProtocol': 'tcp',
   'IpRanges': [],
   'Ipv6Ranges': [],
   'PrefixListIds': [],
   'ToPort': 8080,
   'UserIdGroupPairs': [{'Description': 'tf logan private alb',
     'GroupId': 'sg-02c83117dfa0674b3',
     'UserId': '369096419186'}]}],
 'OwnerId': '369096419186',
 'GroupId': 'sg-04df0e7fdf83a8632',
 'IpPermissionsEgress': [{'IpProtocol': '-1',
   'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
   'Ipv6Ranges': [],
   'PrefixListIds': [],
   'UserIdGroupPairs': []}],
 'Tags': [{'Key': 'Name', 'Value': 'tf_sharevpc_prod_logansite_ecs_task_sg'},
  {'Key': 'terraform', 'Value': 'y'}],
 'VpcId': 'vpc-026ddb1dc52fc3b25'}

In [49]:
alb.describe_load_balancers(LoadBalancerArns=["arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5"])["LoadBalancers"][0]

{'LoadBalancerArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5',
 'DNSName': 'sharesvc-logan-server-alb-963753978.ap-northeast-1.elb.amazonaws.com',
 'CanonicalHostedZoneId': 'Z14GRHDCWA56QT',
 'CreatedTime': datetime.datetime(2020, 1, 7, 8, 43, 42, 880000, tzinfo=tzutc()),
 'LoadBalancerName': 'sharesvc-logan-server-alb',
 'Scheme': 'internet-facing',
 'VpcId': 'vpc-026ddb1dc52fc3b25',
 'State': {'Code': 'active'},
 'Type': 'application',
 'AvailabilityZones': [{'ZoneName': 'ap-northeast-1a',
   'SubnetId': 'subnet-002afc707ec330cd0',
   'LoadBalancerAddresses': []},
  {'ZoneName': 'ap-northeast-1c',
   'SubnetId': 'subnet-00e6b2dd0f34b2e9b',
   'LoadBalancerAddresses': []},
  {'ZoneName': 'ap-northeast-1d',
   'SubnetId': 'subnet-089cdfee95289f367',
   'LoadBalancerAddresses': []}],
 'SecurityGroups': ['sg-0a7f4c1e88f6fb844'],
 'IpAddressType': 'ipv4'}

In [29]:
albs = alb.describe_load_balancers(LoadBalancerArns=["arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-private-alb/9c82d15debe62f2b"])

In [31]:
albs["LoadBalancers"][0]

{'LoadBalancerArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-private-alb/9c82d15debe62f2b',
 'DNSName': 'internal-sharesvc-logan-private-alb-268623527.ap-northeast-1.elb.amazonaws.com',
 'CanonicalHostedZoneId': 'Z14GRHDCWA56QT',
 'CreatedTime': datetime.datetime(2020, 1, 15, 7, 13, 10, 220000, tzinfo=tzutc()),
 'LoadBalancerName': 'sharesvc-logan-private-alb',
 'Scheme': 'internal',
 'VpcId': 'vpc-026ddb1dc52fc3b25',
 'State': {'Code': 'active'},
 'Type': 'application',
 'AvailabilityZones': [{'ZoneName': 'ap-northeast-1d',
   'SubnetId': 'subnet-033b4aac0951ecde8',
   'LoadBalancerAddresses': []},
  {'ZoneName': 'ap-northeast-1a',
   'SubnetId': 'subnet-09d59cc1d56f7e705',
   'LoadBalancerAddresses': []},
  {'ZoneName': 'ap-northeast-1c',
   'SubnetId': 'subnet-0e4de7af47f34419f',
   'LoadBalancerAddresses': []}],
 'SecurityGroups': ['sg-02c83117dfa0674b3'],
 'IpAddressType': 'ipv4'}

In [50]:
alb.describe_listeners(LoadBalancerArn="arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5")["Listeners"]

[{'ListenerArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5/29749e7fdd0db5b7',
  'LoadBalancerArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5',
  'Port': 443,
  'Protocol': 'HTTPS',
  'Certificates': [{'CertificateArn': 'arn:aws:acm:ap-northeast-1:369096419186:certificate/c5b2a39a-3224-4dac-90e4-2e574432bbb0'}],
  'SslPolicy': 'ELBSecurityPolicy-2016-08',
  'DefaultActions': [{'Type': 'fixed-response',
    'Order': 1,
    'FixedResponseConfig': {'MessageBody': 'Illegal request',
     'StatusCode': '403',
     'ContentType': 'text/plain'}}]}]

In [33]:
listeners = alb.describe_listeners(LoadBalancerArn="arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-private-alb/9c82d15debe62f2b")

In [36]:
listeners["Listeners"]

[{'ListenerArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener/app/sharesvc-logan-private-alb/9c82d15debe62f2b/a6202a3010dc62a8',
  'LoadBalancerArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:loadbalancer/app/sharesvc-logan-private-alb/9c82d15debe62f2b',
  'Port': 80,
  'Protocol': 'HTTP',
  'DefaultActions': [{'Type': 'fixed-response',
    'Order': 1,
    'FixedResponseConfig': {'StatusCode': '403',
     'ContentType': 'text/plain'}}]}]

In [52]:
json.dumps(alb.describe_rules(ListenerArn="arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5/29749e7fdd0db5b7")["Rules"])

'[{"RuleArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener-rule/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5/29749e7fdd0db5b7/e535823c3a11b357", "Priority": "2", "Conditions": [{"Field": "path-pattern", "Values": ["/logan/upload.json"], "PathPatternConfig": {"Values": ["/logan/upload.json"]}}], "Actions": [{"Type": "forward", "TargetGroupArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logan-server-tg/7f3592cb6275b4b3", "Order": 1, "ForwardConfig": {"TargetGroups": [{"TargetGroupArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logan-server-tg/7f3592cb6275b4b3", "Weight": 1}], "TargetGroupStickinessConfig": {"Enabled": false}}}], "IsDefault": false}, {"RuleArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener-rule/app/sharesvc-logan-server-alb/48a1b4bc6ffb3de5/29749e7fdd0db5b7/c00fb7b1de7a5c66", "Priority": "3", "Conditions": [{"Field": "path-pattern", "Values": ["/logan/we

In [39]:
rules = alb.describe_rules(ListenerArn="arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener/app/sharesvc-logan-private-alb/9c82d15debe62f2b/a6202a3010dc62a8")

In [56]:
buf = io.StringIO()
yaml.dump(rules["Rules"], buf)

In [60]:
buf.seek(0)
print(buf.read())

- RuleArn: arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener-rule/app/sharesvc-logan-private-alb/9c82d15debe62f2b/a6202a3010dc62a8/d68fdfc6e014c390
  Priority: '3'
  Conditions:
  - Field: path-pattern
    Values:
    - /logan/*
    PathPatternConfig:
      Values:
      - /logan/*
  Actions:
  - Type: forward
    TargetGroupArn: arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-loganserver-p-tg/fb3441a6fab3b3af
    Order: 1
    ForwardConfig:
      TargetGroups:
      - TargetGroupArn: arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-loganserver-p-tg/fb3441a6fab3b3af
        Weight: 1
      TargetGroupStickinessConfig:
        Enabled: false
  IsDefault: false
- RuleArn: arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener-rule/app/sharesvc-logan-private-alb/9c82d15debe62f2b/a6202a3010dc62a8/377d2b9729f0ec23
  Priority: '4'
  Conditions:
  - Field: path-pattern
    Values:
    - /*
    PathPatternCon

In [42]:
()

'[{"RuleArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener-rule/app/sharesvc-logan-private-alb/9c82d15debe62f2b/a6202a3010dc62a8/d68fdfc6e014c390", "Priority": "3", "Conditions": [{"Field": "path-pattern", "Values": ["/logan/*"], "PathPatternConfig": {"Values": ["/logan/*"]}}], "Actions": [{"Type": "forward", "TargetGroupArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-loganserver-p-tg/fb3441a6fab3b3af", "Order": 1, "ForwardConfig": {"TargetGroups": [{"TargetGroupArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-loganserver-p-tg/fb3441a6fab3b3af", "Weight": 1}], "TargetGroupStickinessConfig": {"Enabled": false}}}], "IsDefault": false}, {"RuleArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:listener-rule/app/sharesvc-logan-private-alb/9c82d15debe62f2b/a6202a3010dc62a8/377d2b9729f0ec23", "Priority": "4", "Conditions": [{"Field": "path-pattern", "Values": ["/*"], "PathPatternConfig"

In [43]:
json.dumps({'apiVersion': 'elbv2.aws.infra/v1alpha1', 'kind': 'LoadBalancer', 'metadata': {'Name': 'sharesvc-logan-private-alb-debug', 'Subnets': ['subnet-0e4de7af47f34419f', 'subnet-09d59cc1d56f7e705', 'subnet-033b4aac0951ecde8'], 'SecurityGroups': ['sg-02c83117dfa0674b3'], 'Scheme': 'internal', 'Tags': {'message': 'managed by https://git.toolsfdg.net/devops/devops-infra', 'team': 'devops', 'biz': 'common-infra'}, 'Type': 'application', 'IpAddressType': 'ipv4'}, 'spec': {'Listeners': [{'Protocol': 'HTTP', 'Port': 80, 'DefaultActions': [{'Type': 'fixed-response', 'Order': 1, 'FixedResponseConfig': {'ContentType': 'text/plain', 'StatusCode': '403'}}], 'Rules': [{'Priority': 3, 'Conditions': [{'Field': 'path-pattern', 'PathPatternConfig': {'Values': ['/logan/*']}}], 'Actions': [{'Type': 'forward', 'Order': 1, 'ForwardConfig': {'TargetGroups': [{'TargetGroupArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-loganserver-p-tg/fb3441a6fab3b3af', 'Weight': 1}], 'TargetGroupStickinessConfig': {'Enabled': False}}}]}, {'Priority': 4, 'Conditions': [{'Field': 'path-pattern', 'PathPatternConfig': {'Values': ['/*']}}], 'Actions': [{'Type': 'forward', 'Order': 1, 'ForwardConfig': {'TargetGroups': [{'TargetGroupArn': 'arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-logansite-tg/0170f6bb3fbe6595', 'Weight': 1}], 'TargetGroupStickinessConfig': {'Enabled': False}}}]}]}]}})

'{"apiVersion": "elbv2.aws.infra/v1alpha1", "kind": "LoadBalancer", "metadata": {"Name": "sharesvc-logan-private-alb-debug", "Subnets": ["subnet-0e4de7af47f34419f", "subnet-09d59cc1d56f7e705", "subnet-033b4aac0951ecde8"], "SecurityGroups": ["sg-02c83117dfa0674b3"], "Scheme": "internal", "Tags": {"message": "managed by https://git.toolsfdg.net/devops/devops-infra", "team": "devops", "biz": "common-infra"}, "Type": "application", "IpAddressType": "ipv4"}, "spec": {"Listeners": [{"Protocol": "HTTP", "Port": 80, "DefaultActions": [{"Type": "fixed-response", "Order": 1, "FixedResponseConfig": {"ContentType": "text/plain", "StatusCode": "403"}}], "Rules": [{"Priority": 3, "Conditions": [{"Field": "path-pattern", "PathPatternConfig": {"Values": ["/logan/*"]}}], "Actions": [{"Type": "forward", "Order": 1, "ForwardConfig": {"TargetGroups": [{"TargetGroupArn": "arn:aws:elasticloadbalancing:ap-northeast-1:369096419186:targetgroup/sharesvc-loganserver-p-tg/fb3441a6fab3b3af", "Weight": 1}], "Target